# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).


In [ ]:
#add api key to url
url = 'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID='+weather_api_key 

#create a list to store data
city_weather_check = []

#Start format for printing results
print("Begining Data Retrieval")
print("-----------------------------")

#Create a for loop to gather data on each city
for city in cities:
    city_weather_url = url + "&q=" + city

    try:
        response = requests.get(city_weather_url)
        response_json = response.json()
        city_weather_lat = response_json["coord"]["lat"]
        city_weather_lng = response_json["coord"]["lon"]
        city_weather_weather = response_json["main"]["temp"]
        city_weather_temp_max = response_json["main"]["temp_max"]
        city_weather_humidity = response_json["main"]["humidity"]
        city_weather_windspeed = response_json["wind"]["speed"]
        city_weather_cloudiness = response_json["clouds"]["all"]
        city_weather_country = response_json["sys"]["country"]
        city_weather_date = response_json["dt"]
        city_weather_check.append({"City": city,
                                 "Latitude":city_weather_lat,
                                 "Longitude":city_weather_lng,
                                 "City Weather":city_weather_weather,
                                  "Max Temp":city_weather_temp_max,
                                  "Humidity": city_weather_humidity,
                                  "Cloudiness": city_weather_cloudiness,
                                  "Windspeed": city_weather_windspeed,
                                  "Country": city_weather_country,
                                  "Date": city_weather_date})
        print(f"{city}, Processed")
    except:
        print(f"{city}, Not Found. Skipping...")
        pass
#finsih format for printing results   
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

In [ ]:
#Check to make sure list was filled with data
city_weather_check

In [ ]:
#check to see if json was correct
response = requests.get(city_weather_url)
response_json = response.json()
response_json

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
#create a new data frame for city weather data
city_weather_df = pd.DataFrame(city_weather_check)
city_weather_df

In [ ]:
#Export data to a csv and locate it
city_weather_df.to_csv("../resources/city_weather_csv")

In [ ]:
#Get data frame summary
city_weather_df.describe()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#Reduce data frame to only include city with less than 100% humidity
clean_city_data = city_weather_df.loc[city_weather_df["Humidity"] <100]
clean_city_data

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
#Create a plot for City Latitude vs Max Temp
temp_x = clean_city_data["Latitude"]
temp_y = clean_city_data["Max Temp"]
temp_colors = ["orange"]

temp_scatter = plt.scatter(temp_x,temp_y, c= temp_colors, edgecolors = "black")
plt.title("City Latitude vs Max Temp")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.grid()
plt.show()
plt.savefig("../resources/lat_vs_temp.png")

### The graph performs a parabola and shows as you move away from the equator the temperature drops significantly.

## Latitude vs. Humidity Plot

In [ ]:
#Create a plot for City Latitude vs Humidity
humidity_x = clean_city_data["Latitude"]
humidity_y = clean_city_data["Humidity"]
humidity_colors = ["red"]

humidity_scatter = plt.scatter(humidity_x, humidity_y, c= humidity_colors, edgecolors = "yellow")
plt.title("City Latitude vs Humidity (%)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid()
plt.show()
plt.savefig("../resources/lat_vs_humidity.png")

### Humidity looks to stay at relatively high percentage thoughout the longitudes dipping down a little bit in certian areas of latitude

## Latitude vs. Cloudiness Plot

In [ ]:
#Create a plot for City Latitude vs Cloudiness
cloudiness_x = clean_city_data["Latitude"]
cloudiness_y = clean_city_data["Cloudiness"]
cloudiness_colors = ["lightblue"]

cloudiness_scatter = plt.scatter(cloudiness_x, cloudiness_y, c= cloudiness_colors, edgecolors = "grey")
plt.title("City Latitude vs Cloudiness (%)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness(%)")
plt.grid()
plt.show()
plt.savefig("../resources/lat_vs_cloudiness.png")

### There looks to be really little coralation with cloudiness percentage and it doesn't matter where you are for how cloudy it is

## Latitude vs. Wind Speed Plot

In [ ]:
#Create a plot for City Latitude vs Wind Speed
windspeed_x = clean_city_data["Latitude"]
windspeed_y = clean_city_data["Windspeed"]
windspeed_colors = ["grey"]

windspeed_scatter = plt.scatter(windspeed_x, windspeed_y, c= windspeed_colors, edgecolors = "black")
plt.title("City Latitude vs Wind Speed (mph)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.grid()
plt.show()
plt.savefig("../resources/lat_vs_windspeed.png")

### Throughout the world it looks like windspeed will stay below 15 mph with a few exceptions here and there

## Linear Regression

In [ ]:
#Separate the two hemispheres into their own data frame
northern_hemisphere_df = clean_city_data.loc[clean_city_data["Latitude"] >= 0]
southern_hemisphere_df = clean_city_data.loc[clean_city_data["Latitude"] <= 0]
southern_hemisphere_df

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#Declare x and y values
north_temp_x_lr = northern_hemisphere_df["Latitude"]
north_temp_y_lr = northern_hemisphere_df["Max Temp"]
north_temp_lr_colors = ["orange"]

#Create scatter plot
north_temp_scatter = plt.scatter(north_temp_x_lr, north_temp_y_lr, c= north_temp_lr_colors, edgecolors = "black")
plt.title("Northern Hemisphere City Latitude vs Max Temp")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")


# Formula for y= mx+b
#north_temp_formula = northern_hemisphere_df["Max Temp"] * north_temp_lr.slope + north_temp_lr.intercept
north_temp_lr = linregress(north_temp_x_lr,north_temp_y_lr)
line_eq = "y = " + str(round(north_temp_lr.slope,2)) + "x + " + str(round(north_temp_lr.intercept,2))
north_temp_slope = north_temp_lr.slope
b_north_temp = north_temp_lr.intercept
x_north_temp = north_temp_x_lr
north_temp_formula = x_north_temp * north_temp_slope + b_north_temp

#Plot linear regression
plt.annotate(line_eq,(0,40),fontsize=15,color="red")
plt.plot(x_north_temp, north_temp_formula)
plt.savefig("../resources/northern_lat_vs_temp.png")

### In the northern hemisphere the temperature goes down are you move up in latitude

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#Declare x and y values
south_temp_x_lr = southern_hemisphere_df["Latitude"]
south_temp_y_lr = southern_hemisphere_df["Max Temp"]
south_temp_lr_colors = ["orange"]

#Label the plot
south_temp_scatter = plt.scatter(south_temp_x_lr,south_temp_y_lr, c= south_temp_lr_colors, edgecolors = "black")
plt.title("Southern Hemisphere City Latitude vs Humidity (%)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
south_temp_lr = linregress(south_temp_x_lr,south_temp_y_lr)

#Formula for y = mx + b
line_eq2 = "y = " + str(round(south_temp_lr.slope,2)) + "x + " + str(round(south_temp_lr.intercept,2))
south_temp_slope = south_temp_lr.slope
b_south_temp = south_temp_lr.intercept
x_south_temp = south_temp_x_lr
south_temp_formula = x_south_temp * south_temp_slope + b_south_temp

#Plot linear regression
plt.annotate(line_eq2,(-50,80),fontsize=15,color="red")
plt.plot(x_south_temp, south_temp_formula)
plt.savefig("../resources/southern_lat_vs_temp.png")

### As you move further down the southern hemishpere the temperature gets colder

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Declare x and y values
north_humidity_x_lr = northern_hemisphere_df["Latitude"]
north_humidity_y_lr = northern_hemisphere_df["Humidity"]
north_humidity_lr_colors = ["orange"]

#Label the plot
north_humidity_scatter = plt.scatter(north_temp_x_lr, north_temp_y_lr, c= north_temp_lr_colors, edgecolors = "green")
plt.title("Northern Hemisphere City Latitude vs Humidity (%)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

#Set up formula
linregress(north_humidity_x_lr,north_humidity_y_lr)

north_humidity_lr = linregress(north_humidity_x_lr,north_humidity_y_lr)
line_eq3 = "y = " + str(round(north_humidity_lr.slope,2)) + "x + " + str(round(north_humidity_lr.intercept,2))
north_humidity_slope = north_humidity_lr.slope
b_north_humidity = north_humidity_lr.intercept
x_north_humidity = north_humidity_x_lr
north_humidity_formula = x_north_humidity * north_humidity_slope + b_north_humidity

#Plot linear regression
plt.annotate(line_eq3,(0,40),fontsize=15,color="red")
plt.plot(x_north_humidity, north_humidity_formula)
plt.savefig("../resources/northern_lat_vs_humidity.png")

### As you move furthern north the humidity will start to go down

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Declare x and y values
south_humidity_x_lr = southern_hemisphere_df["Latitude"]
south_humidity_y_lr = southern_hemisphere_df["Humidity"]
south_humidity_lr_colors = ["orange"]

#Label the plot
south_humidity_scatter = plt.scatter(south_humidity_x_lr, south_humidity_y_lr, c= south_humidity_lr_colors, edgecolors = "green")
plt.title("Southern Hemisphere City Latitude vs Humidity (%)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
south_humidity_lr = linregress(south_humidity_x_lr,south_humidity_y_lr)

#Formula for y = mx + b
line_eq4 = "y = " + str(round(south_humidity_lr.slope,2)) + "x + " + str(round(south_humidity_lr.intercept,2))
south_humidity_slope = south_humidity_lr.slope
b_south_humidity = south_humidity_lr.intercept
x_south_humidity = south_humidity_x_lr
south_humidity_formula = x_south_humidity * south_humidity_slope + b_south_humidity

#Plot linear regression
plt.annotate(line_eq4,(-55,20),fontsize=15,color="red")
plt.plot(x_south_humidity, south_humidity_formula)
plt.savefig("../resources/southern_lat_vs_humidity.png")

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#Declare x and y values
north_cloudiness_x_lr = northern_hemisphere_df["Latitude"]
north_cloudiness_y_lr = northern_hemisphere_df["Cloudiness"]
north_cloudiness_lr_colors = ["orange"]

#Label the plot
north_cloudiness_scatter = plt.scatter(north_cloudiness_x_lr, north_cloudiness_y_lr, c= north_cloudiness_lr_colors, edgecolors = "green")
plt.title("Northern Hemisphere City Latitude vs Cloudiness (%)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
north_cloudiness_lr = linregress(north_cloudiness_x_lr,north_cloudiness_y_lr)

#Set up formula
line_eq5 = "y = " + str(round(north_cloudiness_lr.slope,2)) + "x + " + str(round(north_cloudiness_lr.intercept,2))
north_cloudiness_slope = north_cloudiness_lr.slope
b_north_cloudiness = north_cloudiness_lr.intercept
x_north_cloudiness = north_cloudiness_x_lr
north_cloudiness_formula = x_north_cloudiness * north_cloudiness_slope + b_north_cloudiness

#Plot linear regression
plt.annotate(line_eq5,(45,25),fontsize=15,color="red")
plt.plot(x_north_cloudiness, north_cloudiness_formula)
plt.savefig("../resources/northern_lat_vs_cloudiness.png")

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#Declare x and y values
south_cloudiness_x_lr = southern_hemisphere_df["Latitude"]
south_cloudiness_y_lr = southern_hemisphere_df["Cloudiness"]
south_cloudiness_lr_colors = ["orange"]

#Label the plot
south_cloudiness_scatter = plt.scatter(south_cloudiness_x_lr, south_cloudiness_y_lr, c= south_cloudiness_lr_colors, edgecolors = "green")
plt.title("Southern Hemisphere City Latitude vs Cloudiness (%)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
south_cloudiness_lr = linregress(south_cloudiness_x_lr, south_cloudiness_y_lr)

#Set up formula
line_eq6 = "y = " + str(round(south_cloudiness_lr.slope,2)) + "x + " + str(round(south_cloudiness_lr.intercept,2))
south_cloudiness_slope = south_cloudiness_lr.slope
b_south_cloudiness = south_cloudiness_lr.intercept
x_south_cloudiness = south_cloudiness_x_lr
south_cloudiness_formula = x_south_cloudiness * south_cloudiness_slope + b_south_cloudiness

#Plot linear regression
plt.annotate(line_eq6,(-55,20),fontsize=15,color="black")
plt.plot(x_south_cloudiness, south_cloudiness_formula)
plt.savefig("../resources/southern_lat_vs_cloudiness.png")

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#Declare x and y values
north_windspeed_x_lr = northern_hemisphere_df["Latitude"]
north_windspeed_y_lr = northern_hemisphere_df["Windspeed"]
north_windspeed_lr_colors = ["orange"]

#Label plot
north_windspeed_scatter = plt.scatter(north_windspeed_x_lr, north_windspeed_y_lr, c= north_windspeed_lr_colors, edgecolors = "green")
plt.title("Northern Hemisphere City Latitude vs Wind Speed (mph)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
north_windspeed_lr = linregress(north_windspeed_x_lr,north_windspeed_y_lr)

#Set up formula
line_eq7 = "y = " + str(round(north_windspeed_lr.slope,2)) + "x + " + str(round(north_windspeed_lr.intercept,2))
north_windspeed_slope = north_windspeed_lr.slope
b_north_windspeed = north_windspeed_lr.intercept
x_north_windspeed = north_windspeed_x_lr
north_windspeed_formula = x_north_windspeed * north_windspeed_slope + b_north_windspeed

#Plot linear regression
plt.annotate(line_eq7,(45,21),fontsize=15,color="red")
plt.plot(x_north_windspeed, north_windspeed_formula)
plt.savefig("../resources/northern_lat_vs_windspeed.png")

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#Declare x and y values
south_windspeed_x_lr = southern_hemisphere_df["Latitude"]
south_windspeed_y_lr = southern_hemisphere_df["Windspeed"]
south_windspeed_lr_colors = ["orange"]

#Label plot
south_cloudiness_scatter = plt.scatter(south_windspeed_x_lr, south_windspeed_y_lr, c= south_windspeed_lr_colors, edgecolors = "green")
plt.title("Southern Hemisphere City Latitude vs Wind Speed (mph)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

#Set up formula
south_windspeed_lr = linregress(south_windspeed_x_lr,south_windspeed_y_lr)
line_eq8 = "y = " + str(round(south_windspeed_lr.slope,2)) + "x + " + str(round(south_windspeed_lr.intercept,2))
south_windspeed_slope = south_windspeed_lr.slope
b_south_windspeed = south_windspeed_lr.intercept
x_south_windspeed = south_windspeed_x_lr
south_windspeed_formula = x_south_windspeed * south_windspeed_slope + b_south_windspeed

#Plot linear regression
plt.annotate(line_eq8,(-25,30),fontsize=15,color="red")
plt.plot(x_south_windspeed, south_windspeed_formula)
plt.savefig("../resources/southern_lat_vs_windspeed.png")